In [1]:
push!(LOAD_PATH, ".");
using Knet; using KnetONNX;

In [2]:
#rnn_graph = KnetONNX.ONNXtoGraph("rnn.onnx");

In [2]:
g = KnetONNX.ONNXtoGraph("branch1.onnx");

In [3]:
KnetONNX.Display_Graph(g)

(op1) Gemm
	input1: input
	input2: linear1.weight
	input3: linear1.bias
	output1: 5
(op2) Gemm
	input1: input
	input2: linear2.weight
	input3: linear2.bias
	output1: 6
(op3) Concat
	input1: 5
	input2: 6
	output1: 7


In [67]:
struct ModelLayer
    inputs #list of strings
    layer # a KnetLayer
    outputs #list of strings
end

function ModelLayer(node, g)
    ModelLayer(node.input, KnetONNX.node_to_layer(node, g), node.output)
end

(m::ModelLayer)(x) = m.layer(x)

function get_ModelLayers(g)
    ModelLayers = []
    for node in g.node; push!(ModelLayers, ModelLayer(node, g)); end
    return ModelLayers
end

get_ModelLayers (generic function with 1 method)

In [68]:
function GetNextLayers(k::KnetModel, tensor)
    next_layers = []
    for layer in k.model_layers
        if tensor in layer.inputs; push!(next_layers, layer); end
    end
    next_layers #returns a list of ModelLayer s
end

GetNextLayers (generic function with 1 method)

In [91]:
mutable struct KnetModel
    tensors
    model_layers
    model_inputs
    model_outputs
end

function KnetModel(g::KnetONNX.Types.Graph)
    tensors = TensorDict(g)
    model_layers = get_ModelLayers(g)
    model_inputs = [i.name for i in g.input]
    model_outputs = [o.name for o in g.output]
    KnetModel(tensors, model_layers, model_inputs, model_outputs)
end

function (m::KnetModel)(x)
    # init tensor_queue with inputs of the model
    tensor_queue = []
    for input in m.model_inputs; push!(tensor_queue, input); end
    m.tensors[m.model_inputs[1]] = x   #assume there is only one input, (fix later if necessary)

    
    # for each tensor in the queue get the layers they make use of them
    for tensor in tensor_queue
        
        tensor_value = m.tensors[tensor]
        next_layers = GetNextLayers(m, tensor)
        
        for layer in next_layers
        
            # get input tensors
            the_inputs = Dict()
            for input in layer.inputs; the_inputs[input] = m.tensors[input]; end
            if Nothing in values(the_inputs); println("oops"); continue; end
            
            # apply the layer to the inputs
            out = 
            
        end
    end
    
end
    
    
function TensorDict(g::KnetONNX.Types.Graph)
    tensors = Dict()
    for node in g.node
        for input in node.input; tensors[input] = Nothing; end
        for output in node.output; tensors[output] = Nothing; end
    end
    for (node, value) in g.initializer; tensors[node] = value; end
    tensors
end

model = KnetModel(g);
dummy_input = randn(200,50)
model(dummy_input)

KnetONNX.KnetLayers.Linear(KnetONNX.KnetLayers.Multiply(Float32[-0.049053647 -0.05550183 -0.038912855 -0.0022516847 -0.0699565 -0.02747596 0.04775971 0.03521367 -0.06663775 0.011922814 0.07034188 0.046238743 0.038060255 0.019840106 -0.024370432 0.017596334 -0.041143656 -0.049612887 0.05929979 -0.026945155 0.06614628 0.043471508 -0.019131254 -0.056025513 0.05166451 0.064044245 -0.021754146 0.00706736 0.045949146 -0.041748486 -0.050298616 -0.067902595 -0.01128123 0.057819493 0.05829718 -0.016047195 -0.046636283 0.054276206 0.0074062645 -0.06467962 -0.039986495 -0.03088038 0.016101502 0.03837625 -0.01992543 0.033096217 0.057534717 0.04679592 -0.07021903 -0.016974743 -0.009291485 0.016961731 -0.04780944 -0.0024604723 -0.03818541 -0.062560305 0.05930505 0.019551836 -0.06541194 0.018551476 0.040524878 0.022290565 0.04153224 -0.054065146 -0.040052574 0.020890065 0.058747984 -0.04542388 -0.044662613 -0.0513803 0.0060104057 0.014698923 0.023437262 -0.03483213 0.05937349 0.055022754 -0.057616085

 -0.0221266 0.027692102 0.04310409 -9.752065e-5 0.060700886 -0.01172984 -0.06607069 0.028545946 0.009248033 -0.004284419 0.021914996 -0.060464665 0.024818242 0.011244535 -0.054893054 -0.042478453 0.048489124 0.01806879 0.016335629 0.038943544 0.018363915 -0.01547543 -0.015969947 -0.048058234 0.070125096 0.06043566 -0.04386941 0.024250925 0.013611205 -0.034741748 0.024176463 0.049857475 -0.06656482 -0.049699366 -0.05126576 -0.034695018 -0.056742456 0.06951707 -0.056643773 0.040794164 0.05287911 -0.032211684 0.003840752 -0.043448884 -0.053794622 0.015555672 -0.041733786 0.01097808 -0.016504135 0.024799459 0.011131793 0.01636853 0.001363039 0.06998206 -0.0388058 -0.06313506 -0.056711413 -0.06097454 0.013433956 -0.049841 -0.062485024 0.02469948 -0.0085860565 0.06942708 -0.026515149 0.02223958 0.0034550726 -0.057808116 -0.037964433 -0.049510024 0.03573198 0.06409607 -0.03746151 -0.058352374 -0.021589562 0.0170056 0.059293203 0.027678728 0.017688192 -0.02596546 0.039917283 0.054032803 -0.022

006033413 0.05201476 0.06393067 0.038522273 0.01570294 -0.019288067 -0.005217634 0.044354744 -0.054581158 -0.030277092 -0.005469829 -0.059955277 -0.015150182 0.003421031 -0.016930852 0.026740648 -0.033984058 -0.05937584 0.053863473 -0.058980145 -0.06101469 0.03982763 0.033407725 -0.001313217 0.029412083 0.047193214 0.016327709 -0.024787769 0.015468538 0.03821592 -0.028075196 0.053070642 -0.010947231 0.015256681 -0.018514253 -0.030779853 0.05259116 0.012231246 -0.0141328685 0.031513073 -0.025222816 0.0689644 -0.03146001 -0.05883701 0.028968118 -0.0094514415 0.06244319 -0.05777883 0.027760185 -0.031216241 0.006106317 0.0123541355 -0.025658708 -0.016107447 -0.024266344 0.017781831 0.038464658 -0.01502417 0.03207025 -0.029686723 0.011470586 0.0603149 0.044453323 0.036909983 -0.026477959 0.0097200945 -0.022033133 -0.013931572 -0.010908313 -0.030710056 0.0063784122 0.026176438 -0.0063975155 0.025564536 0.011192054 0.046705097 -0.030861106 0.02896288 -0.048081625 -0.03894052 -0.058038794 -0.0

KnetONNX.KnetLayers.Linear(KnetONNX.KnetLayers.Multiply(Float32[0.005453445 0.053825937 0.0070003197 -0.05755798 0.023658939 0.03410653 -0.04534541 -0.014507301 0.036803886 0.06021381 -0.046003863 0.061707444 0.009590827 0.059873305 0.040049605 -0.042700913 0.031878054 0.007181257 0.02560395 -0.02902142 0.03545937 -0.055180766 -0.025085807 -0.016830731 0.015407629 0.06931726 -0.05048444 -0.012123071 -0.058627114 0.062125318 0.02198115 -0.06725842 -0.04148217 0.06533774 -0.04602523 0.001975298 -0.068145335 0.030155428 -0.061218165 -0.010403924 0.0008779466 -0.04885635 -0.037641454 0.0053219944 0.07017667 -0.010011241 0.009847201 -0.02648665 0.041774265 -0.057214558 -0.068908535 0.008798398 -0.061997063 0.024854973 0.037602715 0.031166129 0.0071517453 0.060021527 -0.0001007542 -0.05668437 0.017277949 -0.06670192 0.012918659 0.020977363 0.023971528 -0.002111569 0.046005704 0.06712922 -0.02629568 0.01655896 -0.03387161 0.00955762 -0.042773142 -0.034210943 -4.2676926e-5 -0.029580422 0.04024

 0.009490803 -0.05371702 -0.0041045547 -0.009812083 0.033155106 -0.01417369 0.011067934 -0.06669243 -0.0494788 0.054306097 -0.028230347 -0.06829349 -0.018093321 -0.018961549 -0.05899072 0.05309666 0.069797985 0.03032703 -0.053500857 0.03151396 0.01463794 -0.00084628165 0.06010995 0.04525245 -0.016706523 -0.0022039115 0.022720143 -0.02701931 0.06238287 -0.010612685 -0.016058028 0.022723593 -0.01472234 -0.06617089 -0.04637205 -0.03936467 -0.00041689724 0.05536174 -0.038048726 -0.049593963 -0.008009464 -0.00120873 0.0154976845 -0.008154161 0.06553727 -0.003449738 -0.047602177 -0.068588175 0.027191103 -0.012369849 0.024716236 -0.04131778 0.05566012 0.05274336 0.03470985 -0.047075782 -0.018964034 0.034758665 0.06548219 0.041193597 -0.06052381 -0.0022231862 -0.0671158 -0.00041535497 0.033265278 -0.0366529 0.0020634308 -0.029412441 0.069512 -0.0035195723 0.06991183 0.049264714 0.041795217 0.030711941 -0.058138773 -0.046799593 0.04595139 -0.058364682 0.03852088 -0.034940675 -0.054111496 0.0659

0.017988637 0.022890702 -0.01172575 0.014129877 0.0063630864 0.0024386644 -0.058737967 -0.06397395 -0.0589158 0.06448563 -0.0142420195 0.0024353564 -0.05800248 0.061377756 -0.046964765 0.019848183 0.023926623 -0.06553302 -0.023403201 -0.0320858 0.002790153 -0.045328964 0.032103427 0.03455226 0.03306114 0.060097836 -0.041515574 -0.010945901 -0.056403384 0.03364478 -0.042437766 -0.007449135 0.034712873 -0.014510486 0.03720744 0.06655807 0.019659586 0.053220905 0.053265415 0.05714526 0.012414463 -0.06911506 0.063483484 0.03435929 0.04024174 0.04449469 -3.580004e-5 0.050734386 -0.047207195 0.03644879 0.018198662 0.027639009 0.02550982 -0.019550946 -0.012812164 0.056049593 0.0005232692 -0.05582739 -0.06408894 0.054783158 -0.06937743 -0.04490993 0.0357045 -0.049297687 0.030156754 -0.0030643046 0.038453385 0.03548488; 0.059552453 0.046319827 0.02282115 0.05553288 -0.061536357 -0.020095915 0.019402243 -0.044680603 -0.025330883 0.0025673062 -0.039287608 -0.06953413 0.0097818 0.04496993 -0.02474

 0.06566077 0.0543418 0.06323115 -0.043908127 0.026841499 0.029563472 0.07034869 0.006254196 0.058160163 0.05747702 -0.015266355 -0.00879785 -0.062017757 -0.020119365 -0.048149988 0.0020538867 0.02030959 0.06190134 -0.052491482 -0.00916516 -0.0039262623 0.067171924 -0.03489906 0.0127487555 0.051222786 0.008735962 0.027343705 0.03843183; -0.041339308 -0.030989196 -0.060964525 -0.02353442 0.03660369 0.06658187 -0.054631323 -0.019326504 -0.01983269 -0.010362215 0.016562715 0.015993789 0.031125233 0.063799255 0.058310814 -0.028554011 -0.015180621 -0.008759685 -0.0020286813 0.04790934 0.064296626 -0.04873064 -0.025625352 0.060737737 -0.001568757 0.021746859 -0.055695098 0.057097144 0.0043677464 -0.028773166 -0.057692245 0.05131305 -0.028668422 0.006355338 0.029336452 -0.0089939535 -0.010612711 -0.021301135 -0.062423177 0.06751902 -0.0060670003 0.03662225 0.025827557 -0.0011660606 -0.012723662 0.036477946 -0.029091831 0.064940594 -0.069846764 0.028824992 0.03509722 -0.0023584813 -0.011825174

0.030244611 -0.02261202 0.060244583 0.03242576 -0.016686715 0.037921153 -0.05078923 -0.04625719 -0.018593866 -0.02537248 -0.04558444 -0.03916132 -0.0433004 0.013357803 -0.044606686 0.0050563067 -0.048612952 0.03403846 -0.04457278 0.028140597 0.06591082 0.06343127 -0.004213758 0.036420576 -0.06730577 -0.030247953 0.06852531 -0.020829253 -0.018157132 -0.01416849 -0.06028468 -0.024999794 -0.066124536 -0.052843105 -0.013354786 0.012493432 0.018030278 0.06851695 0.022151686 0.010542773 0.048373617 0.011506125 0.015986234 0.012889713 -0.026600994 0.012538262 -0.046309903 0.04524439 -0.05954105 -0.055512417 0.051054694 -0.027303614 0.039918616 -0.058797833 0.039323434 0.032186396 0.007194191 0.054198228 0.030036964 -0.0042222813 -0.035442375 -0.05062887 0.05799716 -0.045380414 -0.012644611 -0.056397434 0.03462801 0.060161985 -0.06673939 -0.048377924 0.010053568 0.0282517 0.035501897 0.03295356 0.044049487 -0.011516962 0.065254234 0.03476861 0.044981427 0.06557479 -0.034981783 0.054080717 0.01

-0.048535824 -0.06187016 -0.02753067 0.0013265535 -0.015115809 0.06557395 0.02447255 -0.04663786 -0.05583705 0.03867013 0.04969099 -0.03836751 0.01966583 0.021661915 -0.0063335076 -0.04632386 0.06114263 -0.050698783 0.06823469 -0.057027858 -0.00466232 0.004276514 -0.03318224 0.023543432 0.06078545 0.047040403 0.0014907867 -0.0072958916 -0.064556345 -0.031406365 -0.011870898 0.0422434 -0.0051402375 -0.066457964 0.037880227 0.03154829 0.07021833 0.034454748 -0.00343813 -0.05896528 0.046020843 0.00083265454 0.020302266 0.06266724 0.020767637 0.032114103 -0.044411786 0.03284149 0.061404206 0.026664354 -0.0002817586 0.011277378 0.0009976476 -0.04384081 -0.04516904 -0.009948291 0.044349127 -0.004373282 0.00077127665 0.063335605 -0.04425823 -0.0626391 0.033076353 0.01666034 0.051723324 0.00571952 -0.021104068 0.0038479045 -0.059112094 -0.04763834 -0.028986279 -0.06042815 0.037258193 0.04475402 -0.051547293 -0.04888158 -0.05450672 0.05845342 0.03289801 -0.06967389 -0.066231 0.029278964 -0.0255

46599984 -0.040304735 -0.06429485 -0.011401977 -0.011822522 0.064630695 0.019993782 -0.012399115 -0.004086703 -0.059861258 -0.0140484385 0.038718745 -0.065840825 0.04212384 0.03637436 -0.013573274 0.025449589 0.041499972 -0.04183199 0.04104618 0.004768707 -0.009482343 -0.034063034 0.017457746 0.044430606 0.022707537 0.046975397 -0.024845384 0.026979774 0.00049900264 0.038308024 -0.012831222 0.044006318 0.013722621 0.06668133 0.039458483 0.0011973977 -0.014456268 0.04882008 -0.036262803 -0.032140117 0.057195775 0.045798406 -0.01991152 -0.060380142 0.0149379 0.068052806 0.048416838 0.011258446 -0.014333554 0.038930587 -0.06542501 0.018617421 0.03466686 -0.0302536 -0.026438534 -0.030735701 0.014951564 0.067159854 -0.04897646 -0.019108977 -0.015717287 -0.024055425 -0.03177409 -0.026154269 -0.070012726 -0.017010123 0.05206736 0.049552187 0.023055151 0.02978877 -0.039208718 0.006083533 -0.06807212 0.0013526157 -0.007378049 -0.039134167 -0.06199383 0.03723841 -0.04467138 0.045104735 -0.047940